#### Purpose
This notebook initializes the Silver layer by reading curated Bronze Delta data
without modification. It establishes the entry point for all downstream Silver
transformations.

#### Scope
- Read Bronze Delta tables at dataset root level
- Validate schema and record availability
- Preserve partitioning columns for downstream processing


###### Upstream Dependency
- Bronze Delta table:
  - Container: `bronze`
  - Dataset: `raw_data/data_set`
  - Format: Delta
  - Partitions: `source_year`, `source_quarter`

In [0]:
storage_account_name = "flightdatastorage"
storage_account_key = ""
container_name = "raw"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    storage_account_key
)

In [0]:
BRONZE_PATH = (
    "wasbs://bronze@flightdatastorage.blob.core.windows.net/"
    "raw_data/data_set/"
)

###### Read Bronze Delta Table

Read Bronze Delta table from the dataset root path.
All partitions and columns are loaded as-is to ensure schema parity

In [0]:
df_bronze = (
    spark.read
    .format("delta")
    .load(BRONZE_PATH)
)


###### Bronze Read Verification

In [0]:
df_bronze.printSchema()
df_bronze.count()


root
 |-- ITIN_ID: string (nullable = true)
 |-- MKT_ID: string (nullable = true)
 |-- MARKET_COUPONS: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- QUARTER: string (nullable = true)
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: string (nullable = true)
 |-- ORIGIN_CITY_MARKET_ID: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_COUNTRY: string (nullable = true)
 |-- ORIGIN_STATE_FIPS: string (nullable = true)
 |-- ORIGIN_STATE_ABR: string (nullable = true)
 |-- ORIGIN_STATE_NM: string (nullable = true)
 |-- ORIGIN_WAC: string (nullable = true)
 |-- DEST_AIRPORT_ID: string (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: string (nullable = true)
 |-- DEST_CITY_MARKET_ID: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_COUNTRY: string (nullable = true)
 |-- DEST_STATE_FIPS: string (nullable = true)
 |-- DEST_STATE_ABR: string (nullable = true)
 |-- DEST_STATE_NM: string (nullable = true)
 |-- 

56930991

###### Column Standardization
Standardize column names to `snake_case` and apply
business-friendly naming conventions required by the Silver layer.

In [0]:
from pyspark.sql import functions as F

rename_map = {
    "YEAR": "year",
    "QUARTER": "quarter",
    "MARKET_FARE": "market_fare_usd",
    "PASSENGERS": "passenger_cnt",
    "MARKET_DISTANCE": "market_distance_miles",
    "MARKET_MILES_FLOWN": "market_miles_flown",
    "NONSTOP_MILES": "nonstop_miles"
}

df_silver_renamed = df_bronze

for old_col, new_col in rename_map.items():
    if old_col in df_silver_renamed.columns:
        df_silver_renamed = df_silver_renamed.withColumnRenamed(
            old_col, new_col
        )

In [0]:
df_silver_casted \
    .filter(F.col("source_year") == "2024") \
    .filter(F.col("source_quarter") == "04") \
    .select(
        F.count("*").alias("total"),
        F.sum(F.col("ITIN_ID").isNull().cast("int")).alias("itin_nulls"),
        F.sum(F.col("MKT_ID").isNull().cast("int")).alias("mkt_nulls"),
        F.sum(F.col("market_fare_usd").isNull().cast("int")).alias("fare_nulls"),
        F.sum(F.col("passenger_cnt").isNull().cast("int")).alias("pax_nulls")
    ) \
    .show()


+--------+----------+---------+----------+---------+
|   total|itin_nulls|mkt_nulls|fare_nulls|pax_nulls|
+--------+----------+---------+----------+---------+
|24351696|  17054668| 17054668|      4514|     4514|
+--------+----------+---------+----------+---------+



###### Type Casting & Schema Enforcement
Cast standardized columns to Silver-approved data types


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    IntegerType, LongType, DecimalType, TimestampType
)

df_silver_casted = (
    df_silver_renamed
    .withColumn("year", F.col("year").cast("int"))
    .withColumn("quarter", F.col("quarter").cast("int"))
    .withColumn("passenger_cnt", F.col("passenger_cnt").cast("double").cast("int"))
    .withColumn("market_fare_usd", F.col("market_fare_usd").cast("decimal(10,2)"))
    .withColumn("market_distance_miles", F.col("market_distance_miles").cast("double").cast("int"))
    .withColumn("market_miles_flown", F.col("market_miles_flown").cast("double").cast("bigint"))
    .withColumn("nonstop_miles", F.col("nonstop_miles").cast("double").cast("int"))
    .withColumn("ingestion_ts", F.col("ingestion_ts").cast("timestamp"))
)

In [0]:
df_silver_casted.select("source_quarter").distinct().show()

+--------------+
|source_quarter|
+--------------+
|            01|
|            03|
|            02|
|            04|
+--------------+



###### Data Quality Enforcement
Remove invalid records and
standardize key attributes required for downstream reliability.


In [0]:
df_silver_dq = (
    df_silver_casted
    .filter(F.col("ITIN_ID").isNotNull())
    .filter(F.col("MKT_ID").isNotNull())
    .filter(F.col("source_year").isNotNull())
    .filter(F.col("source_quarter").isNotNull())
    .filter(F.col("market_fare_usd") >= 0)
    .filter(F.col("passenger_cnt") >= 0)
)


In [0]:
df_silver_casted \
    .filter(F.col("source_year") == "2025") \
    .filter(F.col("source_quarter") == "01") \
    .select(
        F.count("*").alias("total"),
        F.sum(F.col("ITIN_ID").isNull().cast("int")).alias("itin_nulls"),
        F.sum(F.col("MKT_ID").isNull().cast("int")).alias("mkt_nulls"),
        F.sum(F.col("market_fare_usd").isNull().cast("int")).alias("fare_nulls"),
        F.sum(F.col("passenger_cnt").isNull().cast("int")).alias("pax_nulls")
    ) \
    .show()


+-------+----------+---------+----------+---------+
|  total|itin_nulls|mkt_nulls|fare_nulls|pax_nulls|
+-------+----------+---------+----------+---------+
|7297028|         0|        0|         0|        0|
+-------+----------+---------+----------+---------+



In [0]:
df_silver_dq.select("source_quarter").distinct().show()

+--------------+
|source_quarter|
+--------------+
|            02|
|            03|
|            04|
|            01|
+--------------+



In [0]:
df_silver_dq = (
    df_silver_dq.withColumn("passenger_cnt", F.col("passenger_cnt").cast("int"))
.withColumn("market_distance_miles", F.col("market_distance_miles").cast("int"))
.withColumn("nonstop_miles", F.col("nonstop_miles").cast("int"))
)

In [0]:
df_silver_flags = (
    df_silver_dq
    .withColumn(
        "tk_carrier_change_flg",
        F.expr("try_cast(TK_CARRIER_CHANGE as int)")
    )
    .withColumn(
        "op_carrier_change_flg",
        F.expr("try_cast(OP_CARRIER_CHANGE as int)")
    )
)


In [0]:
df_silver_enriched = (
    df_silver_flags
    .withColumn(
        "is_nonstop",
        F.col("nonstop_miles") == F.col("market_distance_miles")
    )
    .withColumn(
        "has_carrier_change",
        (F.col("tk_carrier_change_flg") == 1) |
        (F.col("op_carrier_change_flg") == 1)
    )
)


###### Type Casting, Schema Enforcement & Quality Validation

In [0]:
df_silver_enriched.printSchema()


root
 |-- ITIN_ID: string (nullable = true)
 |-- MKT_ID: string (nullable = true)
 |-- MARKET_COUPONS: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: string (nullable = true)
 |-- ORIGIN_AIRPORT_SEQ_ID: string (nullable = true)
 |-- ORIGIN_CITY_MARKET_ID: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_COUNTRY: string (nullable = true)
 |-- ORIGIN_STATE_FIPS: string (nullable = true)
 |-- ORIGIN_STATE_ABR: string (nullable = true)
 |-- ORIGIN_STATE_NM: string (nullable = true)
 |-- ORIGIN_WAC: string (nullable = true)
 |-- DEST_AIRPORT_ID: string (nullable = true)
 |-- DEST_AIRPORT_SEQ_ID: string (nullable = true)
 |-- DEST_CITY_MARKET_ID: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_COUNTRY: string (nullable = true)
 |-- DEST_STATE_FIPS: string (nullable = true)
 |-- DEST_STATE_ABR: string (nullable = true)
 |-- DEST_STATE_NM: string (nullable = true)
 |-

In [0]:
df_silver_enriched.count()


39876323

###### Derived Columns
Create derived attributes without aggregating or changing the grain of the dataset.


In [0]:
df_silver_enriched = (
    df_silver_flags
    .withColumn(
        "is_nonstop",
        F.col("nonstop_miles") == F.col("market_distance_miles")
    )
    .withColumn(
        "has_carrier_change",
        (F.col("tk_carrier_change_flg") == 1) |
        (F.col("op_carrier_change_flg") == 1)
    )
)

In [0]:
df_silver_enriched = (
    df_silver_flags
    .withColumn(
        "is_nonstop",
        F.col("nonstop_miles") == F.col("market_distance_miles")
    )
    .withColumn(
        "has_carrier_change",
        F.when(
            (F.col("tk_carrier_change_flg") == 1) |
            (F.col("op_carrier_change_flg") == 1),
            F.lit(True)
        ).otherwise(F.lit(False))
    )
)


In [0]:
df_silver_enriched.select(
    "is_nonstop",
    "has_carrier_change"
).groupBy(
    "is_nonstop",
    "has_carrier_change"
).count().show()


DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-30 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

+----------+------------------+--------+
|is_nonstop|has_carrier_change|   count|
+----------+------------------+--------+
|      true|             false|23107068|
|     false|             false|16769255|
+----------+------------------+--------+



###### Persist Silver Layer
Curated Silver dataset to Delta format with partitioning and write to silver table

In [0]:
SILVER_PATH = (
    "wasbs://silver@flightdatastorage.blob.core.windows.net/"
    "flight_market/"
)

(
    df_silver_enriched
    .write
    .format("delta")
    .mode("append")
    .partitionBy("source_year", "source_quarter")
    .save(SILVER_PATH)
)


DEBUG:ThreadMonitor:Logging python thread stack frames for MainThread and py4j threads:
DEBUG:ThreadMonitor:Logging Thread-31 (run) stack frames:
  File "/usr/lib/python3.12/threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/databricks/python/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 521, in run
    self.wait_for_commands()
  File "/databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 593, in wait_for_commands
    command = smart_decode(self.stream.readline())[:-1]
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)

DEBUG:ThreadMonitor:Logging Th

###### Validate

In [0]:
spark.read.format("delta").load(SILVER_PATH).count()

72455618